In [ ]:
# import libraries
from datetime import date
import numpy as np
import requests
import pandas as pd
import os
import io
from datetime import datetime, timedelta
import functions # specific module for additional functions for this code
from glob import glob
pd.set_option('display.width', 2000)
import xml.etree.ElementTree as ET

In [ ]:
url = 'https://sanctionslistservice.ofac.treas.gov/api/PublicationPreview/exports/SDN_ADVANCED.XML'

response = requests.get(url)
xml_content = response.content
root = ET.fromstring(xml_content)

# Namespace handling
ns = {'ns': 'https://sanctionslistservice.ofac.treas.gov/api/PublicationPreview/exports/ADVANCED_XML'}

In [ ]:
# from now on, there will be many cells similar to the following, as this is the structure of the OFAC XML file: 
# it is essentially an entire relational database, with many encoding tables all linked by cross-referenced keys to the most important tables that will be extracted later in this code
# so I'll comment once for all cause I'm too lazy and it's really straightforward :)

# find the 'AliasTypeValues' section
alias_types = root.find('.//ns:AliasTypeValues', ns)

# extract data into a list of dictionaries
data_alias_types = []
for alias_type in alias_types:
    id = alias_type.attrib['ID']
    text = alias_type.text
    data_alias_types.append({'AliasTypeID': id, 'AliasType_text': text})

# create the DataFrame
df_alias_type = pd.DataFrame(data_alias_types)

# %%
idreg_types = root.find('.//ns:IDRegDocTypeValues', ns)

data_reg_types = []
for reg_type in idreg_types:
    id = reg_type.attrib['ID']
    text = reg_type.text
    data_reg_types.append({'IDRegDocTypeID': id, 'IDRegDocType_text': text})

df_reg_type = pd.DataFrame(data_reg_types)

# %%
idregdocuments_types = root.find('.//ns:IDRegDocuments', ns)

data_regdocuments_types = []
for r in idregdocuments_types:
    id = r.attrib['ID']
    reg_doc_id = r.attrib['IDRegDocTypeID']
    identity_id = r.attrib['IdentityID']
    id_registration_no = r.find('ns:IDRegistrationNo', ns).text if r.find('ns:IDRegistrationNo', ns) is not None else None
    data_regdocuments_types.append({'IDRegDocument_id': id, 'IDRegDocTypeID': reg_doc_id, 'IdentityID_id': identity_id, 'IDRegistrationNo': id_registration_no})

df_regdocuments_type = pd.DataFrame(data_regdocuments_types)

# %%
# that's actually the first example of the structure of the XML: 
# I'm merging the DataFrames df_regdocuments_type and df_reg_type on the key 'IDRegDocTypeID' to have a complete table
id_reg_ofac = pd.merge(df_regdocuments_type, df_reg_type, on='IDRegDocTypeID', how='right')

# %%
area_codes = root.find('.//ns:AreaCodeValues', ns)

data_area_code = []
for area_code in area_codes:
    id = area_code.attrib['ID']
    country_id = area_code.attrib['CountryID']
    description = area_code.attrib['Description']
    text = area_code.text
    data_area_code.append({'area_code_id': id, 'CountryID': country_id, 'description': description, 'iso2': text})

df_area_code = pd.DataFrame(data_area_code)

# %%
detail_ref = root.find('.//ns:DetailReferenceValues', ns)

data_detail_ref = []
for d in detail_ref:
    id = d.attrib['ID']
    text = d.text
    data_detail_ref.append({'DetailReferenceID': id, 'DetailReference_text': text})

df_detail_ref = pd.DataFrame(data_detail_ref)

# %%
detail_type = root.find('.//ns:DetailTypeValues', ns)

data_detail_type = []
for d in detail_type:
    id = d.attrib['ID']
    text = d.text
    data_detail_type.append({'DetailTypeID': id, 'DetailType_text': text})

df_detail_type = pd.DataFrame(data_detail_type)

# %%
feature_type = root.find('.//ns:FeatureTypeValues', ns)

data_feature_type = []
for b in feature_type:
    id = b.attrib['ID']
    feature_type_group_id = b.attrib['FeatureTypeGroupID']
    text = b.text
    data_feature_type.append({'FeatureTypeID': id, 'FeatureTypeGroupID': feature_type_group_id, 'FeatureType_text': text})

df_feature_type = pd.DataFrame(data_feature_type)

# %%
date_type = root.find('.//ns:IDRegDocDateTypeValues', ns)

data_date_type = []
for b in date_type:
    id = b.attrib['ID']
    text = b.text
    data_date_type.append({'date_type_id': id, 'date_type_text': text})

df_date_type = pd.DataFrame(data_date_type)

# %%
legal_basis = root.find('.//ns:LegalBasisValues', ns)

data_legal_basis = []
for b in legal_basis:
    id = b.attrib['ID']
    legal_basis_short_ref = b.attrib['LegalBasisShortRef']
    legal_basis_type_id = 'Unknown'
    sanction_prg_id = b.attrib['SanctionsProgramID']
    text = b.text
    data_legal_basis.append({'LegalBasisID': id, 'LegalBasiShortRef': legal_basis_short_ref, 'LegalBasisTypeID': legal_basis_type_id, 'SanctionsProgramID': sanction_prg_id, 'LegalBasis_text': text})

df_legal_basis = pd.DataFrame(data_legal_basis)

# %%
list_values = root.find('.//ns:ListValues', ns)

data_list_values = []
for b in list_values:
    id = b.attrib['ID']
    text = b.text
    data_list_values.append({'ListID': id, 'List_text': text})

df_list_values = pd.DataFrame(data_list_values)

# %%
loc_part_type = root.find('.//ns:LocPartTypeValues', ns)

data_loc_part_type = []
for b in loc_part_type:
    id = b.attrib['ID']
    text = b.text
    data_loc_part_type.append({'LocPartTypeID': id, 'LocPartType_text': text})

df_loc_part_type = pd.DataFrame(data_loc_part_type)

# %%
name_part_type = root.find('.//ns:NamePartTypeValues', ns)

data_name_part_type = []
for b in name_part_type:
    id = b.attrib['ID']
    text = b.text
    data_name_part_type.append({'NamePartTypeID': id, 'NamePartType_text': text})

df_name_part_type = pd.DataFrame(data_name_part_type)

# %%
org_val = root.find('.//ns:OrganisationValues', ns)

data_org_val = []
for b in org_val:
    id = b.attrib['ID']
    text = b.text
    data_org_val.append({'org_id': id, 'org_text': text})

df_org_val = pd.DataFrame(data_org_val)

# %%
party_sub_type = root.find('.//ns:PartySubTypeValues', ns)

data_party_sub_type = []
for b in party_sub_type:
    id = b.attrib['ID']
    party_type_id = b.attrib['PartyTypeID']
    text = b.text
    data_party_sub_type.append({'PartySubTypeID': id, 'PartyTypeID': party_type_id, 'PartySubType_text': text})

df_party_sub_type = pd.DataFrame(data_party_sub_type)

# %%
party_type = root.find('.//ns:PartyTypeValues', ns)

data_party_type = []
for b in party_type:
    id = b.attrib['ID']
    text = b.text
    data_party_type.append({'PartyTypeID': id, 'PartyType_text': text})

df_party_type = pd.DataFrame(data_party_type)

# %%
relation_qual = root.find('.//ns:RelationQualityValues', ns)

data_relation_qual = []
for b in relation_qual:
    id = b.attrib['ID']
    text = b.text
    data_relation_qual.append({'relation_quality_id': id, 'relation_quality_text': text})

df_relation_qual = pd.DataFrame(data_relation_qual)

# %%
relation_type = root.find('.//ns:RelationTypeValues', ns)

data_relation_type = []
for b in relation_type:
    id = b.attrib['ID']
    text = b.text
    data_relation_type.append({'relation_type_id': id, 'relation_type_text': text})

df_relation_type = pd.DataFrame(data_relation_type)

# %%
reliability = root.find('.//ns:ReliabilityValues', ns)

data_reliability = []
for b in reliability:
    id = b.attrib['ID']
    text = b.text
    data_reliability.append({'ReliabilityID': id, 'Reliability_text': text})

df_reliability = pd.DataFrame(data_reliability)

# %%
sanction_prg = root.find('.//ns:SanctionsProgramValues', ns)

data_sanction_prg = []
for b in sanction_prg:
    id = b.attrib['ID']
    sub_body_id = b.attrib['SubsidiaryBodyID']
    text = b.text
    data_sanction_prg.append({'SanctionsProgramID': id, 'sub_body_id': sub_body_id, 'SanctionsProgram_text': text})

df_sanction_prg = pd.DataFrame(data_sanction_prg)

# %%
sanction = root.find('.//ns:SanctionsTypeValues', ns)

data_sanction = []
for b in sanction:
    id = b.attrib['ID']
    text = b.text
    data_sanction.append({'SanctionsTypeID': id, 'SanctionsType_text': text})

df_sanction_type = pd.DataFrame(data_sanction)

# %%
validity = root.find('.//ns:ValidityValues', ns)

data_validity = []
for b in validity:
    id = b.attrib['ID']
    text = b.text
    data_validity.append({'validity_id': id, 'validity_text': text})

df_validity = pd.DataFrame(data_validity)

In [ ]:
# 'Locations' is one of the tables I was talking about. It's much deeeper and nested, as you can see from teh code below
locations_section = root.findall('.//ns:Locations/ns:Location', ns)

data_locations = []

for location in locations_section:
    location_id = location.attrib.get('ID')

    area_code_element = location.find('ns:LocationAreaCode', ns)
    area_code_id = area_code_element.attrib.get('AreaCodeID') if area_code_element is not None else None

    country_element = location.find('ns:LocationCountry', ns)
    country_id = country_element.attrib.get('CountryID') if country_element is not None else None
    country_relevance_id = country_element.attrib.get('CountryRelevanceID') if country_element is not None else None

    loc_part_element = location.find('ns:LocationPart', ns)
    loc_part_type_id = loc_part_element.attrib.get('LocPartTypeID') if loc_part_element is not None else None

    loc_part_value_element = location.find('ns:LocationPart/ns:LocationPartValue', ns)
    loc_part_value = loc_part_value_element.find('ns:Value', ns).text if loc_part_value_element is not None else None
    primary = loc_part_value_element.attrib.get('Primary') if loc_part_value_element is not None else None
    loc_part_value_type_id = loc_part_value_element.attrib.get('LocPartValueTypeID') if loc_part_value_element is not None else None
    loc_part_value_status_id = loc_part_value_element.attrib.get('LocPartValueStatusID') if loc_part_value_element is not None else None

    feature_version_element = location.find('ns:FeatureVersionReference', ns)
    feature_version_id = feature_version_element.attrib.get('FeatureVersionID') if feature_version_element is not None else None

    data_locations.append({
        'LocationID': location_id,
        'AreaCodeID': area_code_id,
        'CountryID': country_id,
        'CountryRelevanceID': country_relevance_id,
        'LocPartTypeID': loc_part_type_id,
        'LocPartValue': loc_part_value,
        'Primary': primary,
        'LocPartValueTypeID': loc_part_value_type_id,
        'LocPartValueStatusID': loc_part_value_status_id,
        'FeatureVersionID': feature_version_id
    })

df_locations = pd.DataFrame(data_locations)

In [ ]:
# 'DistinctParties' is the most important table, and the central one of the XML. It contains all the info about the sanctioned subjects themselves 
# it's really nested and if you need more data, you can check directly the orginal file
parties_section = root.findall('.//ns:DistinctParty', ns)

data_parties = []

for party in parties_section:
    fixed_ref = party.attrib.get('FixedRef')
    comment = party.find('ns:Comment', ns).text if party.find('ns:Comment', ns) is not None else None
    profile = party.find('ns:Profile', ns)

    profile_id = profile.attrib.get('ID') if profile is not None else None
    party_subtype_id = profile.attrib.get('PartySubTypeID') if profile is not None else None

    # 'Identity' data extraction
    identities_data = []
    identities = profile.findall('ns:Identity', ns) if profile is not None else []
    for identity in identities:
        identity_id = identity.attrib.get('ID')
        primary = identity.attrib.get('Primary')
        false = identity.attrib.get('False')

        # 'Alias' data extraction
        aliases_data = []
        aliases = identity.findall('ns:Alias', ns)
        for alias in aliases:
            alias_fixed_ref = alias.attrib.get('FixedRef')
            alias_type_id = alias.attrib.get('AliasTypeID')
            alias_primary = alias.attrib.get('Primary')
            alias_low_quality = alias.attrib.get('LowQuality')

            documented_name = alias.find('ns:DocumentedName', ns)
            documented_name_id = documented_name.attrib.get('ID') if documented_name is not None else None
            doc_name_status_id = documented_name.attrib.get('DocNameStatusID') if documented_name is not None else None

            name_parts = documented_name.findall('ns:DocumentedNamePart/ns:NamePartValue', ns) if documented_name is not None else []
            for name_part in name_parts:
                name_part_value = name_part.text
                name_part_group_id = name_part.attrib.get('NamePartGroupID')
                script_id = name_part.attrib.get('ScriptID')
                script_status_id = name_part.attrib.get('ScriptStatusID')
                acronym = name_part.attrib.get('Acronym')

                aliases_data.append({
                    'AliasFixedRef': alias_fixed_ref,
                    'AliasTypeID': alias_type_id,
                    'AliasPrimary': alias_primary,
                    'AliasLowQuality': alias_low_quality,
                    'DocumentedNameID': documented_name_id,
                    'DocNameStatusID': doc_name_status_id,
                    'NamePartValue': name_part_value,
                    'NamePartGroupID': name_part_group_id,
                    'ScriptID': script_id,
                    'ScriptStatusID': script_status_id,
                    'Acronym': acronym
                })

        # 'NamePartGroups' data extraction
        name_part_groups = identity.findall('ns:NamePartGroups/ns:MasterNamePartGroup/ns:NamePartGroup', ns)
        name_part_groups_data = [{'NamePartGroupID': n.attrib.get('ID'), 'NamePartTypeID': n.attrib.get('NamePartTypeID')} for n in name_part_groups]

        identities_data.append({
            'IdentityID': identity_id,
            'Primary': primary,
            'False': false,
            'Aliases': aliases_data,
            'NamePartGroups': name_part_groups_data
        })

    # 'Feature' data extraction
    features_data = []
    features = profile.findall('ns:Feature', ns) if profile is not None else []
    for feature in features:
        feature_id = feature.attrib.get('ID')
        feature_type_id = feature.attrib.get('FeatureTypeID')

        feature_version = feature.find('ns:FeatureVersion', ns)
        feature_version_id = feature_version.attrib.get('ID') if feature_version is not None else None
        reliability_id = feature_version.attrib.get('ReliabilityID') if feature_version is not None else None

        feature_comment = feature_version.find('ns:Comment', ns).text if feature_version.find('ns:Comment', ns) is not None else None

        # 'DatePeriod' data extraction
        date_period = feature_version.find('ns:DatePeriod', ns)
        date_period_data = {}
        if date_period is not None:
            calendar_type_id = date_period.attrib.get('CalendarTypeID')
            year_fixed = date_period.attrib.get('YearFixed')
            month_fixed = date_period.attrib.get('MonthFixed')
            day_fixed = date_period.attrib.get('DayFixed')

            start = date_period.find('ns:Start', ns)
            end = date_period.find('ns:End', ns)

            start_from = start.find('ns:From', ns) if start is not None else None
            start_to = start.find('ns:To', ns) if start is not None else None
            end_from = end.find('ns:From', ns) if end is not None else None
            end_to = end.find('ns:To', ns) if end is not None else None

            start_from_date = (start_from.find('ns:Year', ns).text, start_from.find('ns:Month', ns).text, start_from.find('ns:Day', ns).text) if start_from is not None else (None, None, None)
            start_to_date = (start_to.find('ns:Year', ns).text, start_to.find('ns:Month', ns).text, start_to.find('ns:Day', ns).text) if start_to is not None else (None, None, None)
            end_from_date = (end_from.find('ns:Year', ns).text, end_from.find('ns:Month', ns).text, end_from.find('ns:Day', ns).text) if end_from is not None else (None, None, None)
            end_to_date = (end_to.find('ns:Year', ns).text, end_to.find('ns:Month', ns).text, end_to.find('ns:Day', ns).text) if end_to is not None else (None, None, None)

            date_period_data = {
                'CalendarTypeID': calendar_type_id,
                'YearFixed': year_fixed,
                'MonthFixed': month_fixed,
                'DayFixed': day_fixed,
                'StartFromDate': start_from_date,
                'StartToDate': start_to_date,
                'EndFromDate': end_from_date,
                'EndToDate': end_to_date
            }

        version_detail = feature_version.find('ns:VersionDetail', ns)
        version_detail_type_id = version_detail.attrib.get('DetailTypeID') if version_detail is not None else None
        version_detail_ref_id = version_detail.attrib.get('DetailReferenceID') if version_detail is not None else None

        identity_reference = feature.find('ns:IdentityReference', ns)
        identity_feature_link_type_id = identity_reference.attrib.get('IdentityFeatureLinkTypeID') if identity_reference is not None else None

        version_location = feature_version.find('ns:VersionLocation', ns)
        version_location_id = version_location.attrib.get('LocationID') if version_location is not None else None

        features_data.append({
            'FeatureID': feature_id,
            'FeatureTypeID': feature_type_id,
            'FeatureVersionID': feature_version_id,
            'ReliabilityID': reliability_id,
            'Comment': feature_comment,
            'DatePeriod': date_period_data,
            'VersionDetailTypeID': version_detail_type_id,
            'VersionDetailReferenceID': version_detail_ref_id,
            'IdentityFeatureLinkTypeID': identity_feature_link_type_id,
            'VersionLocationID': version_location_id
        })

    data_parties.append({
        'FixedRef': fixed_ref,
        'Comment': comment,
        'ProfileID': profile_id,
        'PartySubTypeID': party_subtype_id,
        'Identities': identities_data,
        'Features': features_data
    })

df_parties = pd.json_normalize(data_parties, sep='_')

In [ ]:
# 'explode_nested_col' is a function called from the module defined below. For details about the function, please open the module and read the comments
df_parties_exploded = functions.explode_nested_col(df_parties)
df_parties_exploded_all = functions.explode_nested_col(df_parties_exploded)
# df_parties_all = df_parties_tutto
df_parties_all = df_parties_exploded_all.rename(columns={'Identities_Aliases_AliasTypeID': 'AliasTypeID', 'Features_VersionLocationID': 'LocationID', 'Features_DatePeriod.CalendarTypeID': 'CalendarTypeID',
                                                         'Features_VersionDetailReferenceID': 'DetailReferenceID', 'Features_FeatureTypeID' : 'FeatureTypeID', 'Features_VersionDetailTypeID': 'DetailTypeID',
                                                         'Identities_NamePartGroups_NamePartTypeID': 'NamePartTypeID', 'Features_ReliabilityID': 'ReliabilityID'})

In [ ]:
# 'SanctionsEntry' is one the most interesting tables. It contains all the info about the saanction itself (programme, regulation and so on) 
data = []

for sanctions_entry in root.findall('.//ns:SanctionsEntry', ns):
    base_data = {
        'SanctionsEntry_ID': sanctions_entry.attrib.get('ID'),
        'Profile_ID': sanctions_entry.attrib.get('ProfileID'),
        'List_ID': sanctions_entry.attrib.get('ListID')
    }
    
    # estraction of each 'EntryEvent' for each 'SanctionsEntry'
    for entry_event in sanctions_entry.findall('ns:EntryEvent', ns):
        event_data = {
            'EntryEvent_ID': entry_event.attrib.get('ID'),
            'EntryEventType_ID': entry_event.attrib.get('EntryEventTypeID'),
            'LegalBasis_ID': entry_event.attrib.get('LegalBasisID')
        }

        # dealing with 'Comment' and 'Date' 
        comment = entry_event.find('ns:Comment', ns)
        if comment is not None:
            event_data['Event_Comment'] = comment.text

        date = entry_event.find('ns:Date', ns)
        if date is not None:
            event_data['Event_Date_Year'] = date.find('ns:Year', ns).text
            event_data['Event_Date_Month'] = date.find('ns:Month', ns).text
            event_data['Event_Date_Day'] = date.find('ns:Day', ns).text

        # estraction of each 'SanctionsMeasure' for each 'EntryEvent'
        for sanctions_measure in sanctions_entry.findall('ns:SanctionsMeasure', ns):
            measure_data = {
                'SanctionsMeasure_ID': sanctions_measure.attrib.get('ID'),
                'SanctionsType_ID': sanctions_measure.attrib.get('SanctionsTypeID')
            }

            # dealing with 'Comment' and 'DatePeriod'
            comment = sanctions_measure.find('ns:Comment', ns)
            if comment is not None:
                measure_data['Measure_Comment'] = comment.text

            date_period = sanctions_measure.find('ns:DatePeriod', ns)
            if date_period is not None:
                date = date_period.find('ns:Date', ns)
                if date is not None:
                    measure_data['Measure_Date_Year'] = date.find('ns:Year', ns).text
                    measure_data['Measure_Date_Month'] = date.find('ns:Month', ns).text
                    measure_data['Measure_Date_Day'] = date.find('ns:Day', ns).text

            # combine 'base_data', 'event_data' and 'measure_data' in a new unique field
            combined_data = {**base_data, **event_data, **measure_data}
            data.append(combined_data)

df_sanctions_entries = pd.DataFrame(data)
df_sanctions_entries = df_sanctions_entries.rename(columns={'EntryEventType_ID': 'EventTypeID', 'LegalBasis_ID': 'LegalBasisID', 'List_ID': 'ListID',
                                                            'SanctionsType_ID': 'SanctionsTypeID', 'Profile_ID': 'ProfileID'})

In [ ]:
# now it's time to aggregate all the tables extracted so far in a unique DataFrame
# let's start with the smallest DataFrames
merged_df_party_type = pd.merge(df_party_sub_type, df_party_type, on='PartyTypeID', how='left')
merged_df_sanction_prg = pd.merge(df_sanction_prg, df_legal_basis, on='SanctionsProgramID', how='left')

# 'locations' and 'sanctions' dfs
merged_df_area_code = pd.merge(df_locations, df_area_code, on='CountryID', how='left')
merged_df_loc_part_type = pd.merge(merged_df_area_code, df_loc_part_type, on='LocPartTypeID', how='left')
merged_df_legal_basis = pd.merge(df_sanctions_entries, df_legal_basis, on='LegalBasisID', how='left')
merged_df_list_values = pd.merge(merged_df_legal_basis, df_list_values, on='ListID', how='left')
merged_df_sanctions = pd.merge(merged_df_list_values, df_sanction_type, on='SanctionsTypeID', how='left')

# now the final aggregation
merged_df_alias_type = pd.merge(df_parties_all, df_alias_type, on='AliasTypeID', how='left')
merged_df_party_sub_type = pd.merge(merged_df_alias_type, merged_df_party_type, on='PartySubTypeID', how='left')
merged_df_detail_ref = pd.merge(merged_df_party_sub_type, df_detail_ref, on='DetailReferenceID', how='left')
merged_df_detail_type = pd.merge(merged_df_detail_ref, df_detail_type, on='DetailTypeID', how='left')
merged_df_feature_type = pd.merge(merged_df_detail_type, df_feature_type, on='FeatureTypeID', how='left')
merged_df_name_part_type = pd.merge(merged_df_feature_type, df_name_part_type, on='NamePartTypeID', how='left')
merged_df_reliability = pd.merge(merged_df_name_part_type, df_reliability, on='ReliabilityID', how='left')
merged_df_locations_final = pd.merge(merged_df_reliability, merged_df_loc_part_type, on='LocationID', how='left')
merged_df_sanctions_final = pd.merge(merged_df_locations_final, merged_df_sanctions, on='ProfileID', how='left')

merged_df_sanctions_final = pd.merge(merged_df_sanctions_final, id_reg_ofac, left_on='Identities_IdentityID', right_on='IdentityID_id', how='left')

# I'm getting rid of useless dfs to optimize memory usage 
# I'm using Google cola, so in this environment there is no real need for that, I'm just including a section in which you can add all the delete you need
del merged_df_sanctions
del id_reg_ofac

In [ ]:
# now I have to build one col for the listing date, because OFAC split it in three different columns
merged_df_sanctions_final['listing_date'] = merged_df_sanctions_final['Event_Date_Day']+'/'+merged_df_sanctions_final['Event_Date_Month']+'/'+merged_df_sanctions_final['Event_Date_Year']
merged_df_sanctions_final['listing_date'] = pd.to_datetime(merged_df_sanctions_final['listing_date'], format='%d/%m/%Y')

# I create a dictinary for the columns needed. For sake of completeness, I leave here the list of columns name of merged_df_sanctions_final
col_dict = {
        'ProfileID': None,
        #'Features_FeatureID': None,
        #'DetailTypeID': None,
        #'DetailReferenceID': None,
        #'LocationID': None,
        #'Features_DatePeriod.StartFromDate': None,
        #'Features_DatePeriod.StartToDate': None,
        #'Features_DatePeriod.EndFromDate': None,
        #'Features_DatePeriod.EndToDate': None,
        'Identities_Aliases_NamePartValue': None,
        #'NamePartTypeID': None,
        #'AliasType_text': None,
        #'PartyTypeID': None,
        #'PartySubType_text': None,
        'PartyType_text': None,
        #'CalendarType_text': None,
        #'DetailReference_text': None,
        #'DetailType_text': None,
        #'FeatureTypeGroupID': None,
        'FeatureType_text': None,
        'NamePartType_text': None,
        #'Reliability_text': None,
        #'AreaCodeID': None,
        #'CountryID': None,
        #'CountryRelevanceID': None,
        #'LocPartTypeID': None,
        'LocPartValue': None,
        #'LocPartValueTypeID': None,
        #'LocPartValueStatusID': None,
        #'FeatureVersionID': None,
        #'area_code_id': None,
        'description': None,
        'iso2': None,
        #'CountryRelevanceID_text': None,
        'LocPartType_text': None,
        #'LocPartValueStatus_text': None,
        #'SanctionsEntry_ID': None,
        #'ListID': None,
        #'EntryEvent_ID': None,
        #'EventTypeID': None,
        #'LegalBasisID': None,
        #'Event_Comment': None,
        #'Event_Date_Year': None,
        #'Event_Date_Month': None,
        #'Event_Date_Day': None,
        'listing_date': None,
        'SanctionsTypeID': None,
        'Measure_Comment': None,
        #'EventType_text': None,
        'LegalBasiShortRef': None,
        #'LegalBasisTypeID': None,
        #'SanctionsProgramID': None,
        'LegalBasis_text': None,
        'List_text': None,
        'SanctionsType_text': None,
        'IDRegistrationNo': None,
        'IDRegDocType_text': None
}

selected_col = list(col_dict.keys())

df_ofac_analysis = merged_df_sanctions_final[selected_col]
df_ofac_analysis = df_ofac_analysis.drop_duplicates()

In [ ]:
# selection of needed columns
df_ofac_analysis = df_ofac_analysis.rename(columns={'Identities_Aliases_NamePartValue': 'sanctioned_alias'})
df_ofac_analysis = df_ofac_analysis[['ProfileID', 'iso2', 'PartyType_text', 'listing_date', 'LegalBasis_text']]
df_ofac_analysis['sanction_body'] = 'OFAC'

# standardize columns name: 
# everytihng that's referred to the single sanctioned subject has 'sanctioned_' before the col name
# everytihng that's referred to the sanction itself has 'sanction_' before the col name
df_ofac_analysis = df_ofac_analysis.rename(columns={'ProfileID': 'sanctioned_id',
                                                    'iso2': 'sanctioned_country_iso2',
                                                    'PartyType_text': 'sanctioned_type',
                                                    'listing_date': 'sanction_listing_date',
                                                    'LegalBasis_text': 'sanction_text'})
df_ofac_analysis = df_ofac_analysis.drop_duplicates()

In [ ]:
# this cell saves the DataFrame on your google drive (I use Google colab). You can just save he DataFrame on your local directory as well
from google.colab import drive
drive.mount('/content/drive')
df_ofac_analysis.to_csv('/content/drive/My Drive/df_ofac_analysis.csv', sep=';', index=False)